In [1]:
# increase cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import gc
import time
import re
import s2_py as s2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shapefile as shp
import geopandas as gpd
from shapely.geometry import Polygon, mapping, box
from datetime import date, timedelta

In [5]:
ls Data

CA_State/  Processed/  Transmission_Line/  WildFire_2019.json


In [33]:
# import json
# with open('./Data/WildFire_2019.json', 'r') as data_file:
#     json_data = data_file.read().strip()

# wf2019 = json.loads(json_data)

In [62]:
# source: https://www.fire.ca.gov/incidents/2019/
wf2019_df = pd.read_json('./Data/WildFire_2019.json', orient='records')
wf2019_df.shape

(86, 15)

In [63]:
wf2019_df.head()

,AcresBurnedDisplay,AdminUnit,CountiesList,IncidentId,IncidentTypeDisplay,IsActive,Latitude,Location,Longitude,Name,PercentContainedDisplay,StartedDate,UniqueId,UpdatedDate,Url
0,30,CAL FIRE Humbolt-Del Norte Unit,Humboldt,2005,,N,40.618056,"Pilot Ridge Rd & USFS Route 1, southwest of Wi...",-123.675556,Pilot Fire,100%,2019-01-01,6004d3e5-66f8-487a-80a4-689d3852378c,2019-01-15 10:38:00,https://www.fire.ca.gov/incidents/2019/1/1/pil...
1,80,CAL FIRE Nevada-Yuba-Placer Unit,Yuba,2014,,N,39.246780,"off Scott Forbes Rd, 5 miles east of Browns Va...",-121.323990,Scott Fire,100%,2019-06-25,31cbc968-6d9d-412b-8bc3-6846ca398fe2,2019-06-27 08:24:51,https://www.fire.ca.gov/incidents/2019/6/25/sc...
2,30,Palm Springs City Fire,Riverside,2016,,N,0.000000,"Hwy 111 and Tramway Rd, Palm Springs",0.000000,Girasol Fire,100%,2019-04-09,3b5c5d57-f61d-4c7d-a966-9d522c77e8fc,2019-04-10 10:23:00,https://www.fire.ca.gov/incidents/2019/4/9/gir...
3,161,Fresno-Kings County,Kings,2017,,N,35.817780,"Hwy 41 and York Ave, 13 miles south of Avenal ...",-120.097150,York Fire,100%,2019-04-28,85bdd89a-4ce5-4539-a274-23fa2b030e7d,2019-06-24 11:45:00,https://www.fire.ca.gov/incidents/2019/4/28/yo...
4,10,CAL FIRE / Butte County Fire,Butte,2018,,N,39.839580,"Off Hamilton Nord Cana Hwy & Cana Hwy, northea...",-121.957000,Cana Fire,100%,2019-04-30,34abbc65-5bbe-474b-85c8-16d07bb5b503,2019-05-08 08:47:00,https://www.fire.ca.gov/incidents/2019/4/30/ca...


In [64]:
wf2019_df.dtypes

AcresBurnedDisplay          object
AdminUnit                   object
CountiesList                object
IncidentId                   int64
IncidentTypeDisplay         object
IsActive                    object
Latitude                   float64
Location                    object
Longitude                  float64
Name                        object
PercentContainedDisplay     object
StartedDate                 object
UniqueId                    object
UpdatedDate                 object
Url                         object
dtype: object

In [ ]:
wf2019_df['AcresBurnedDisplay'] = pd.to_numeric(wf2019_df.AcresBurnedDisplay)

In [45]:
def latlong_to_s2(x, lvl):
    latlng = s2.S2LatLng.FromDegrees(x[0], x[1])
    cell = s2.S2CellId(latlng)
    return cell.parent(lvl).ToToken()

In [55]:
temp1 = wf2019_df.copy()
temp2 = wf2019_df.copy()
wf2019_df['S2_Cell_ID'] = wf2019_df[['Latitude','Longitude']].apply(latlong_to_s2, axis=1, args=[9])
temp1['S2_Cell_ID'] = temp1[['Latitude','Longitude']].apply(latlong_to_s2, axis=1, args=[10])
temp2['S2_Cell_ID'] = temp2[['Latitude','Longitude']].apply(latlong_to_s2, axis=1, args=[11])
wf2019_stacked_df = wf2019_df.append(temp1).append(temp2)
wf2019_stacked_df.shape

(258, 16)

In [57]:
wf2019_stacked_df.head()

,AcresBurnedDisplay,AdminUnit,CountiesList,IncidentId,IncidentTypeDisplay,IsActive,Latitude,Location,Longitude,Name,PercentContainedDisplay,StartedDate,UniqueId,UpdatedDate,Url,S2_Cell_ID
0,30,CAL FIRE Humbolt-Del Norte Unit,Humboldt,2005,,N,40.618056,"Pilot Ridge Rd & USFS Route 1, southwest of Wi...",-123.675556,Pilot Fire,100%,2019-01-01,6004d3e5-66f8-487a-80a4-689d3852378c,2019-01-15 10:38:00,https://www.fire.ca.gov/incidents/2019/1/1/pil...,54d394
1,80,CAL FIRE Nevada-Yuba-Placer Unit,Yuba,2014,,N,39.246780,"off Scott Forbes Rd, 5 miles east of Browns Va...",-121.323990,Scott Fire,100%,2019-06-25,31cbc968-6d9d-412b-8bc3-6846ca398fe2,2019-06-27 08:24:51,https://www.fire.ca.gov/incidents/2019/6/25/sc...,809b5c
2,30,Palm Springs City Fire,Riverside,2016,,N,0.000000,"Hwy 111 and Tramway Rd, Palm Springs",0.000000,Girasol Fire,100%,2019-04-09,3b5c5d57-f61d-4c7d-a966-9d522c77e8fc,2019-04-10 10:23:00,https://www.fire.ca.gov/incidents/2019/4/9/gir...,100004
3,161,Fresno-Kings County,Kings,2017,,N,35.817780,"Hwy 41 and York Ave, 13 miles south of Avenal ...",-120.097150,York Fire,100%,2019-04-28,85bdd89a-4ce5-4539-a274-23fa2b030e7d,2019-06-24 11:45:00,https://www.fire.ca.gov/incidents/2019/4/28/yo...,80eb54
4,10,CAL FIRE / Butte County Fire,Butte,2018,,N,39.839580,"Off Hamilton Nord Cana Hwy & Cana Hwy, northea...",-121.957000,Cana Fire,100%,2019-04-30,34abbc65-5bbe-474b-85c8-16d07bb5b503,2019-05-08 08:47:00,https://www.fire.ca.gov/incidents/2019/4/30/ca...,8082dc


In [58]:
wf2019_df.AcresBurnedDisplay.describe()

count     86
unique    65
top       30
freq       5
Name: AcresBurnedDisplay, dtype: object